In [ ]:
import numpy as np
import scipy.signal as scisig
import spectrally_regularised_LVMs as srLVMs

from matplotlib import pyplot as plt
import matplotlib
from axs_fixer import fix

MEDIUM_SIZE = 30
BIGGER_SIZE = 40

plt.rc("font", size=MEDIUM_SIZE)  # controls default text sizes
plt.rc("axes", titlesize=MEDIUM_SIZE)  # fontsize of the axes title
plt.rc("axes", labelsize=MEDIUM_SIZE)  # fontsize of the x and y labels
plt.rc("xtick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("ytick", labelsize=MEDIUM_SIZE)  # fontsize of the tick labels
plt.rc("legend", fontsize=MEDIUM_SIZE)  # legend fontsize
plt.rc("figure", titlesize=BIGGER_SIZE)  # fontsize of the figure title
plt.rc("figure", figsize=(16, 10), dpi=600)
plt.rc("savefig", dpi=600, format="pdf")
plt.rc("grid", linestyle="--")

matplotlib.rcParams.update(
    {  # Use mathtext, not LaTeX
        "text.usetex": False,
        # Use the Computer modern font
        "font.family": "serif",
        "font.serif": "cmr10",
        "mathtext.fontset": "cm",
        # Use ASCII minus
        "axes.unicode_minus": False,
    }
)

plt.ioff()

print(f"Package version: {srLVMs.__version__}")

# Example one

In the first example, the PCA objective function is expressed as a objective function for the *spectrally-regularised-LVMs* package using a symbolic representation and through an explicit representation. The PCA objective function is given by
$$ \mathcal{L}_{model} = \mathbb{E}_{\mathbf{x} \sim p(\mathbf{x})} \{ (\mathbf{w}_i^T \mathbf{x})^2 \},$$
where it is assumed that $\mathbf{x}$ is zero-mean.

## Symbolic representation.

In [ ]:
import sympy as sp
import spectrally_regularised_LVMs as srLVMs

# Symbolic cost function implementation
cost_inst = srLVMs.SymbolicCost() 

z, j, n = cost_inst.get_symbolic_parameters()

loss = -1/n * sp.Sum((z[j])**2, (j))

cost_inst.set_cost(loss) 

# Visualise the loss
sp.pretty_print(loss)

# Visualise the properties of the indexed variables
print(z[j].shape, z[j].ranges)
print(j)
print(n) 

## Explicit representation

In [ ]:
import numpy as np
import spectrally_regularised_LVMs as srLVMs

# Explicit cost function implementation
cost_inst = srLVMs.ExplicitCost()

obj = lambda X, w, z: -1 * np.mean((X @ w)**2, axis = 0)
grad = lambda X, w, z: -2 * np.mean((X @ w) * X, axis = 0, 
                                    keepdims=True).T
hess = lambda X, w, z: -2 / X.shape[0] * (X.T @ X)

cost_inst.set_cost(obj)
cost_inst.set_gradient(grad)
cost_inst.set_hessian(hess) 

## Performing parameter estimation

In [ ]:
import numpy as np
import spectrally_regularised_LVMs as srLVMs

# Define a toy signal
x_signal = np.random.randn(10000)

# Define the cost function instance
cost_inst = ...

# Define the model
model_inst = srLVMs.LinearModel(n_sources = 5,
                                cost_instance = cost_inst, 
                                Lw = 256,
                                Lsft = 1) 

# Estimate the model parameters
model_inst.fit(x_signal)

# Obtain the latent representation Z
Z = model_inst.transform(x_signal)

# Obtain the recovered representation of X
X_recon = model_inst.inverse_transform(Z) 

# Example two

In the second example, the negentropy-based objective function common to ICA is used to estimate the model parameters for a signal from the IMS dataset.

## Using the package - a full example for a signal from the IMS dataset (with spectral regularisation)

In [ ]:
import spectrally_regularised_LVMs as srLVMs
import numpy as np
np.random.seed(0)

# Step 1: Load in the time series signal
data_dict = np.loadtxt("./2004.02.17.07.12.39") # This is the IMS dataset signal stored in the '/Examples/' directory in the Github repository
x_signal = data_dict[:, 0]
Fs = 20480

# Step 2: Define the cost function instance
cost_inst = srLVMs.NegentropyCost("exp", {"alpha":1}) # negentropy objective

# Step 3: Define the model
model_inst = srLVMs.LinearModel(n_sources = 10,
                                cost_instance = cost_inst, 
                                Lw = 256,
                                Lsft = 1,
                                sumt_flag=True,
                                verbose = True,
                                organise_by_kurt=True)

# Step 4: Estimate the model parameters
model_inst.fit(x_signal)

# Step 5: Obtain the latent representation Z
Z = model_inst.transform(x_signal)

# Step 6: Obtain the recovered representation of X
X_recon = model_inst.inverse_transform(Z)

## Using the package - a full example for a signal from the IMS dataset (without spectral regularisation)

In [ ]:
import spectrally_regularised_LVMs as srLVMs
import numpy as np
np.random.seed(0)

# Step 1: Load in the time series signal
data_dict = np.loadtxt("./2004.02.17.07.12.39") # This is the IMS dataset signal stored in the '/Examples/' directory in the Github repository
x_signal = data_dict[:, 0]
Fs = 20480

# Step 2: Define the cost function instance
cost_inst = srLVMs.NegentropyCost("exp", {"alpha":1}) # negentropy objective

# Step 3: Define the model
model_inst = srLVMs.LinearModel(n_sources = 10,
                                cost_instance = cost_inst, 
                                Lw = 256,
                                Lsft = 1,
                                sumt_flag=False,
                                alpha_reg=0,
                                verbose = True,
                                organise_by_kurt=True)

# Step 4: Estimate the model parameters
model_inst.fit(x_signal)

# Step 5: Obtain the latent representation Z
Z2 = model_inst.transform(x_signal)

# Step 6: Obtain the recovered representation of X
X_recon = model_inst.inverse_transform(Z2)

## Visualisation - IMS signal spectrum

In [ ]:
colors = ["#00b0f9", "#bfacdc", "#d0bdd5", "#ffa59e", "#dd4c65","#93003a"]
fig, ax = plt.subplots(figsize = (12, 10))

n = len(x_signal)
fft_freq = np.fft.fftfreq(n, 1/Fs)[:n//2]
fft_val = 2/n * np.abs(np.fft.fft(x_signal)[:n//2])
ax.plot(fft_freq, fft_val, lw = 0.5, color = "k", label = "Record 540")
ax.set_xlabel("Frequency (Hz)")
ax.set_ylabel("Magnitude")
ax.grid()
ax.legend()

fig.tight_layout()

# plt.savefig("./signal.pdf")
# plt.savefig("./signal.png")
plt.show()

## Visualisation - Latent source spectra and SES

In [ ]:
colors = ["#00b0f9", "#bfacdc", "#d0bdd5", "#ffa59e", "#dd4c65","#93003a"]

fig, ax = plt.subplots(5, 2, figsize = (20, 24))

n = Z.shape[0]
fft_freq = np.fft.fftfreq(n, 1/Fs)[:n//2]

for cnt in range(5):
    fft_mag = 2/n * np.abs(np.fft.fft(Z[:, cnt])[:n//2])
    fft_mag2 = 2/n * np.abs(np.fft.fft(Z2[:, cnt])[:n//2])

    ax[cnt, 0].plot(fft_freq, 
                    fft_mag2, 
                    color = "r", 
                    linewidth = 0.5, 
                    label = rf"Source {cnt + 1} without $\mathcal{'{L}'}_{'{sr}'}$")
    
    ax[cnt, 1].plot(fft_freq, 
                    fft_mag, 
                    color = "b", 
                    linewidth = 0.5, 
                    label = rf"Source {cnt + 1} with $\mathcal{'{L}'}_{'{sr}'}$")
    # ax[cnt, 1].set_xlim(-10, 2000)

for cnt, axs in enumerate(ax.flatten()):
    axs.set_xlabel("Frequency (Hz)")
    axs.set_ylabel("Magnitude")
    axs.grid()
    axs.legend()
    fix(axs, minor_flag=True, flag_3d=False)

fig.tight_layout()
    
# plt.savefig("./sources.pdf")
# plt.savefig("./sources.png")
plt.show()

# Example three

In this example, different methods of cost function implementation are demonstrated.

## Method one: Symbolically defined objective

In [ ]:
# Define imports
import spectrally_regularised_LVMs as srLVMs
import numpy as np
import sympy as sp
np.random.seed(0)

# Setup general matrices
X = np.random.randn(500, 16)
X -= np.mean(X, axis = 0, keepdims=True)
w = np.random.randn(16, 1)
z = X @ w

# Initialise the cost function instance
cost_inst = srLVMs.SymbolicCost(use_hessian=True,
                                verbose=True,
                                finite_diff_flag=False)

z_sp, j, n = cost_inst.get_symbolic_parameters()

loss = -1/n * sp.Sum((z_sp[j])**2, (j))

cost_inst.set_cost(loss)
display(loss)

# Check that the gradient and Hessian make sense
res_grad = cost_inst.check_gradient(X, w, z, 1e-4)
res_hess = cost_inst.check_hessian(X, w, z, 1e-4)

## Method two: Explicitly defined objective

In [ ]:
# Define imports
import spectrally_regularised_LVMs as srLVMs
import numpy as np
np.random.seed(0)

# Setup general X matrix
X = np.random.randn(500, 16)
X -= np.mean(X, axis = 0, keepdims=True)
w = np.random.randn(16, 1)
z = X @ w

# Initialise the cost function instance
cost_inst = srLVMs.ExplicitCost(use_hessian=True,
                                verbose=True,
                                finite_diff_flag=False)

# Implement the objective function, gradient and Hessian
def loss(X, w, z):
    return -1 * np.mean((X @ w) ** 2, axis=0)

def grad(X, w, z):
    return -2 * np.mean(z * X, axis=0, keepdims=True).T

def hess(X, w, z):
    return -2 / X.shape[0] * (X.T @ X)

# Set the properties
cost_inst.set_cost(loss)
cost_inst.set_gradient(grad)
cost_inst.set_hessian(hess)

# Check that the gradient and Hessian make sense
res_grad = cost_inst.check_gradient(X, w, z, 1e-4)
res_hess = cost_inst.check_hessian(X, w, z, 1e-4)

## Method 3: In-built variance objective

In [ ]:
# Define imports
import spectrally_regularised_LVMs as srLVMs
import numpy as np
np.random.seed(0)

# Setup general matrices
X = np.random.randn(500, 16)
X -= np.mean(X, axis = 0, keepdims=True) # De-mean the data
w = np.random.randn(16, 1)
z = X @ w

# Initialise the cost function instance
cost_inst = srLVMs.VarianceCost(use_hessian=True,
                                verbose=True,
                                finite_diff_flag=False)

# Check that the gradient and Hessian make sense
res_grad = cost_inst.check_gradient(X, w, z, 1e-4)
res_hess = cost_inst.check_hessian(X, w, z, 1e-4)

## Method 3: In-built negentropy objective

In [ ]:
# Define imports
import spectrally_regularised_LVMs as srLVMs
import numpy as np
np.random.seed(0)

# Setup general matrices
X = np.random.randn(500, 16)
X -= np.mean(X, axis = 0, keepdims=True) # De-mean the data
w = np.random.randn(16, 1)
z = X @ w

## Initialise the cost function instance
cost_inst = srLVMs.NegentropyCost(source_name="exp",
                                  source_params={"alpha": 1},
                                  use_approx=False,
                                  use_hessian=True,
                                  verbose = True,
                                  finite_diff_flag=False)

## Check that the gradient and Hessian make sense
res_grad = cost_inst.check_gradient(X, w, z, 1e-4)
res_hess = cost_inst.check_hessian(X, w, z, 1e-4)